In [1]:
import cv2
import numpy as np
import pdb

from Managers import *

In [2]:
stub_image = 255 * np.ones((100, 100, 3), dtype=np.uint8)

# Stoke Edges

Here we use the Laplacian to make the edges darker and flatter median blurred surfaces retain their colour.

In [25]:
def strokeEdges(src:np.array, blurSize=7, edgeSize=5):
    dst = src.copy()
    if blurSize > 2:
        dst = cv2.medianBlur(dst, blurSize)
    
    dst = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
    dst = cv2.Laplacian(dst, cv2.CV_8U, ksize=edgeSize)
    inverseAlpha = (255 - dst) / 255.
    
    channels = cv2.split(src)
    for channel in channels:
        channel[:] = channel * inverseAlpha
    return cv2.merge(channels)

In [26]:
cam = CameraManager('Stroke Edges', cv2.VideoCapture(0), strokeEdges).addCloseCallback()
cam.run()

# Apply a Kernel

In [7]:
def applyKernel(image:np.array, kernel:np.array):
    return cv2.filter2D(image, -1, kernel)

In [8]:
sharp_kernel = np.array([
    [-1, -1, -1],
    [-1,  9, -1],
    [-1, -1, -1],
])

def sharpen(image):
    return applyKernel(image, sharp_kernel)

In [11]:
cam = CameraManager('Sharpen', cv2.VideoCapture(0), sharpen).addCloseCallback()
cam.run()

In [13]:
emboss_kernel = np.array([
    [-2, -1,  0],
    [-1,  0,  1],
    [0,   1,  2],
])


def emboss(image):
    return applyKernel(image, emboss_kernel)

In [14]:
cam = CameraManager('Emboss', cv2.VideoCapture(0), emboss).addCloseCallback()
cam.run()

# Find Contours

The threshold operation can change the original image.

In [33]:
def thresholdImage(image, lower_thresh=127, upper_thresh=255):
    _, thresh = cv2.threshold(image.copy(), lower_thresh, upper_thresh, 0)
    return thresh

In [3]:
cam = CameraManager('Threshold', cv2.VideoCapture(0), thresholdImage).addCloseCallback()
cam.run()

In [4]:
def saturation(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    _, saturation, _ = cv2.split(hsv)
    return saturation

In [5]:
def blue(image):
    blue, _, _ = cv2.split(image)
    return blue

In [6]:
cam = CameraManager('Saturation', cv2.VideoCapture(0), saturation).addCloseCallback()
cam.run()

In [6]:
def threshSaturation(image, lower_thresh=40, upper_thresh=120):
    return thresholdImage(saturation(image), lower_thresh, upper_thresh)

In [8]:
cam = CameraManager('Blue', cv2.VideoCapture(0), threshSaturation).addCloseCallback()
cam.run()

In [6]:
def threshBlue(image, lower_thresh=160, upper_thresh=255):
    return thresholdImage(blue(image), lower_thresh, upper_thresh)

In [38]:
cam = CameraManager('Saturation', cv2.VideoCapture(0), threshBlue).addCloseCallback()
cam.run()

In [8]:
def findBlueContours(image):
    thresh = threshBlue(image)
    contours, hierachy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return cv2.drawContours(image, contours, -1, (255, 0, 0), 2)

In [18]:
cam = CameraManager('Blue Contours', cv2.VideoCapture(0), findBlueContours).addCloseCallback()
cam.run()

# Get Bouding Box Around the Biggest Blue Object

In [13]:
def findBiggestObjectinBox(image, min_size=1000):
    thresh = threshBlue(image)
    
    contours, hier = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)

        if w * h > min_size:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
    return image

In [14]:
cam = CameraManager('Find Blue Contours', cv2.VideoCapture(0), findBiggestObjectinBox).addCloseCallback()
cam.run()

# Get Oriented Bounding Boxes

In [7]:
def calculateBoxArea(points):
    h = np.linalg.norm(points[0] - points[1])
    w = np.linalg.norm(points[1] - points[2])
    return h * w

In [8]:
def findBiggestObjectinOrientedBox(image, min_size=1000):
    thresh = threshBlue(image)
    
    contours, hier = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for c in contours:
        rect = cv2.minAreaRect(c)
        box = cv2.boxPoints(rect)
        box = np.int64(box)
        
        if calculateBoxArea(box) > min_size:
            cv2.drawContours(image, [box], 0, (0, 0, 255), 3)
        
    return image

In [9]:
cam = CameraManager('Find Oriented Blue Contours', cv2.VideoCapture(0), findBiggestObjectinOrientedBox).addCloseCallback()
cam.run()

# Get Enclosing Circles

In [10]:
def findBiggestObjectinEnclosingCircle(image, min_size=1000):
    thresh = threshBlue(image)
    
    contours, hier = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for c in contours:
        (x, y), radius = cv2.minEnclosingCircle(c)
        
        if np.pi * radius**2 > min_size:
            center = (int(x), int(y))
            radius = int(radius)
            image = cv2.circle(image, center, radius, (255, 0, 0), 2)
        
    return image

In [11]:
cam = CameraManager('Find Circle Blue Contours', cv2.VideoCapture(0), findBiggestObjectinEnclosingCircle).addCloseCallback()
cam.run()

# Get Convex Hull

In [12]:
def findBiggestObjectinConvexHull(image, min_size=1000):
    thresh = threshBlue(image)
    
    contours, hier = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for c in contours:
        if cv2.contourArea(c) > min_size:
            hull = cv2.convexHull(c)
            cv2.drawContours(image, [hull], 0, (0, 0, 255), 3)
        
    return image

In [13]:
cam = CameraManager('Find Hull Blue Contours', cv2.VideoCapture(0), findBiggestObjectinConvexHull).addCloseCallback()
cam.run()

# Line Detection

Hough lines has a stride parameter to increase speed, an angle parameter and a threshold for the number of votes on the line in question.

In [82]:
def findAllLines(image, min_length=100, max_gap=5):
    gray  = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 120, 250)
    
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 50, min_length, max_gap)
    for line in lines:
        (x1, y1, x2, y2) = line[0]
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    return image

In [83]:
cam = CameraManager('Find Lines in Canny Edges', cv2.VideoCapture(0), findAllLines).addCloseCallback()
cam.run()

In [76]:
def findLinesInStoke(image, min_length=500, max_gap=5):
    edges = strokeEdges(image)
    gray  = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    canny = cv2.Canny(gray, 120, 250)
    
    lines = cv2.HoughLinesP(canny, 1, np.pi/180, 50, min_length, max_gap)
    for line in lines:
        (x1, y1, x2, y2) = line[0]
        cv2.line(edges, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    return edges

In [77]:
cam = CameraManager('Find Lines in Canny Edges', cv2.VideoCapture(0), findLinesInStoke).addCloseCallback()
cam.run()

# Circle Detection

The param1 determines the canny edge threshold. The param2 determies the thresold for voting. This algorithm tries to find candidates for centers and then tries to determine the best radius for that center. 

In [3]:
def findCircles(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    smooth = cv2.medianBlur(gray, 5)
    
    circles = cv2.HoughCircles(smooth, cv2.HOUGH_GRADIENT, 1, 120, param1=140, param2=50, minRadius=50, maxRadius=1000)
    if not circles is None:
        circles = np.uint16(np.around(circles))

        for circ in circles[0, :]:
            x, y, r = circ
            cv2.circle(image, (x, y), r, (0, 255, 0), 2)
            cv2.circle(image, (x, y), 2, (0, 255, 0), 3)
    
    return image

In [4]:
cam = CameraManager('Find Lines in Canny Edges', cv2.VideoCapture(0), findCircles).addCloseCallback()
cam.run()